In [ ]:
import numpy as np
from numpy.random import default_rng

import pymedphys

# Create two 100 × 100 × 100 dose grids with corresponding axes:

In [ ]:
WIDTH = 200

DOSE_GRID_SHAPE = (WIDTH, WIDTH, WIDTH)

axes_ref = (
    np.arange(-DOSE_GRID_SHAPE[0] + 1, DOSE_GRID_SHAPE[0], 2),
    np.arange(-DOSE_GRID_SHAPE[1] + 1, DOSE_GRID_SHAPE[0], 2),
    np.arange(-DOSE_GRID_SHAPE[2] + 1, DOSE_GRID_SHAPE[0], 2),
)
axes_eval = axes_ref

rng = default_rng()

dose_ref = rng.normal(loc=2, scale=0.04, size=DOSE_GRID_SHAPE)  # 2% RSU
dose_eval = rng.normal(loc=2, scale=0.04, size=DOSE_GRID_SHAPE)

# Gamma

In [ ]:
dose_percent_thresh = 2
dist_mm_thresh = 2

In [ ]:
%%time
gamma = pymedphys.gamma(
    axes_ref,
    dose_ref,
    axes_eval,
    dose_eval,
    dose_percent_thresh,
    dist_mm_thresh,
    interp_fraction=10,
    interp_algo="pymedphys",
)


In [ ]:
%%time

gamma = pymedphys.gamma(
    axes_ref,
    dose_ref,
    axes_eval,
    dose_eval,
    dose_percent_thresh,
    dist_mm_thresh,
    interp_fraction=10,
    interp_algo="econforge",
)

In [ ]:
%%time
gamma = pymedphys.gamma(
    axes_ref,
    dose_ref,
    axes_eval,
    dose_eval,
    dose_percent_thresh,
    dist_mm_thresh,
    interp_fraction=10,
    interp_algo="scipy",
)